# Question 1 - LLM for data engineering

## 1.  Assume you need datasets to train a recommender system that predicts user preferences for products. Use ChatGPT (GPT-4) to find relevant datasets. List the prompt  you used and the corresponding outputs from ChatGPT

- P:
please find  a dataset to train a recommender system that predicts user preferences 
for products

- A:
     To train a recommender system for predicting user preferences for products, you can 
     consider several datasets available online. One popular option is the **Amazon 
     Review Dataset**, which contains a vast collection of reviews, including ratings, 
     product metadata, and other useful features. This dataset has over 233 million 
     product reviews and is suitable for training and testing product recommendation 
     algorithms.

     Another dataset is the **Steam Video Games dataset**, which records user behaviors 
     such as purchases and gameplay, making it ideal for gaming-related recommendation 
     systems.

     You can find these datasets on platforms like Kaggle or other open repositories:
     - Amazon Review Dataset [Amazon Review Dataset] 
    (https://nijianmo.github.io/amazon/index.html)
     - Steam Video Games Dataset [Steam Video Games](https://www.kaggle.com/datasets)

     These datasets will allow you to build models such as collaborative filtering or 
     content-based recommendation systems.

## 2.Use ChatGPT to preprocess a sample from the Movielens-1M dataset for training a recommender system. List the prompts, inputs, and outputs from ChatGPT

- P:
please preprocess a sample from the Movielens-1M dataset for training a 
recommender system

- A:
Here’s how you can preprocess a sample from the **Movielens-1M** dataset for training a recommender system using Python.

### 1. Download the dataset
First, download the **Movielens-1M** dataset from [here](https://grouplens.org/datasets/movielens/1m/). This dataset contains **1 million ratings** of **movies** by **users**, with fields like `user_id`, `movie_id`, `rating`, and `timestamp`.

### 2. Load necessary libraries
You will need `pandas` and `numpy` for data manipulation.

```python
import pandas as pd
import numpy as np
```

### 3. Load the data
The dataset contains three main files: `ratings.dat`, `users.dat`, and `movies.dat`.

```python
# Load ratings dataset
ratings = pd.read_csv('ratings.dat', sep='::', engine='python',
                      names=['user_id', 'movie_id', 'rating', 'timestamp'])

# Load movies dataset
movies = pd.read_csv('movies.dat', sep='::', engine='python',
                     names=['movie_id', 'title', 'genres'])

# Load users dataset
users = pd.read_csv('users.dat', sep='::', engine='python',
                    names=['user_id', 'gender', 'age', 'occupation', 'zip_code'])
```

### 4. Merge datasets
To work with the data, we can merge the three datasets into one based on `user_id` and `movie_id`.

```python
# Merge ratings with movies
data = pd.merge(ratings, movies, on='movie_id')

# Merge with users
data = pd.merge(data, users, on='user_id')
```

### 5. Preprocess the data
We can preprocess the dataset by normalizing, removing missing values, and encoding categorical features.

#### 5.1 Remove timestamp
```python
data = data.drop('timestamp', axis=1)
```

#### 5.2 Convert categorical columns (like `genres` and `gender`) into numerical format
```python
# Encode gender
data['gender'] = data['gender'].map({'F': 0, 'M': 1})

# Split the genres into multiple columns
genres_split = data['genres'].str.get_dummies('|')
data = pd.concat([data, genres_split], axis=1)
data = data.drop('genres', axis=1)
```

#### 5.3 Normalize ratings (optional)
```python
# Normalize the ratings between 0 and 1 if needed
data['rating'] = data['rating'] / 5.0
```

### 6. Sample data for training
You can now sample the dataset for training purposes.

```python
# Take a sample of 10000 rows for training
train_data = data.sample(10000, random_state=42)
```

### 7. Save the preprocessed data
Finally, save the preprocessed data to a file for later use.

```python
train_data.to_csv('movielens_1m_preprocessed.csv', index=False)
```

### 8. Ready for model training
Now you can use this preprocessed dataset to train your recommender system using collaborative filtering, matrix factorization, or deep learning models like **Neural Collaborative Filtering (NCF)**.

This provides a solid starting point for training a recommender system on the **Movielens-1M** dataset. Let me know if you need help building the model!


## 3.Generate a preprocessing script using ChatGPT to format the dataset for a Collaborative Filtering model using Surprise or TensorFlow. Correct any errors in the generated code, use the revised version to preprocess the dataset, and display the first 5 processed entries.

- P:
Give me a preprocessing script to format the dataset for a Collaborative
Filtering model using Surprise or TensorFlow. 

- A:
Here is a Python script to preprocess the **Movielens-1M** dataset for a Collaborative Filtering model using **Surprise** or **TensorFlow**.

### Preprocessing for Surprise or TensorFlow Collaborative Filtering Model

```python
import pandas as pd
from surprise import Dataset, Reader
from surprise.model_selection import train_test_split

# Load ratings.dat
ratings = pd.read_csv('ratings.dat', sep='::', engine='python',
                      names=['user_id', 'movie_id', 'rating', 'timestamp'])

# Drop unnecessary columns
ratings = ratings.drop('timestamp', axis=1)

# Ensure correct data types
ratings['user_id'] = ratings['user_id'].astype(int)
ratings['movie_id'] = ratings['movie_id'].astype(int)
ratings['rating'] = ratings['rating'].astype(float)

# For Surprise Library
def preprocess_for_surprise():
    # Define the rating scale
    reader = Reader(rating_scale=(1, 5))

    # Load the dataset for Surprise from Pandas DataFrame
    surprise_data = Dataset.load_from_df(ratings[['user_id', 'movie_id', 'rating']], reader)

    # Train-Test Split
    trainset, testset = train_test_split(surprise_data, test_size=0.2)

    return trainset, testset

# For TensorFlow Model
def preprocess_for_tensorflow():
    import tensorflow as tf

    # Create user-movie matrix (utility matrix)
    user_movie_matrix = ratings.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)

    # Convert to TensorFlow tensors
    user_ids = tf.convert_to_tensor(user_movie_matrix.index.values)
    movie_ids = tf.convert_to_tensor(user_movie_matrix.columns.values)
    rating_matrix = tf.convert_to_tensor(user_movie_matrix.values)

    return user_ids, movie_ids, rating_matrix

# Example usage
if __name__ == '__main__':
    # Preprocess for Surprise
    trainset, testset = preprocess_for_surprise()

    # Preprocess for TensorFlow
    user_ids, movie_ids, rating_matrix = preprocess_for_tensorflow()

    print("Surprise dataset and TensorFlow tensors are ready!")
```

### Explanation:
1. **For Surprise:**
   - We load the dataset into a Pandas DataFrame, drop unnecessary columns, and define the `Reader` object with a rating scale of 1 to 5.
   - The dataset is loaded into the **Surprise** format using `Dataset.load_from_df()`, and we split it into training and test sets using `train_test_split()`.

2. **For TensorFlow:**
   - The data is converted into a user-movie interaction matrix (utility matrix) with users as rows and movies as columns. Missing ratings are filled with 0.
   - The utility matrix is then converted into TensorFlow tensors for further processing in a neural-based collaborative filtering model.

### Next Steps:
- **Surprise**: You can use algorithms like **SVD**, **KNN** from the Surprise library to build the collaborative filtering model.
- **TensorFlow**: The tensor data is ready to be passed into a deep learning model, such as a **Neural Collaborative Filtering (NCF)** model.

Let me know if you need further guidance on building the model itself!

- P:
Check if there is any errors in the generated code

- A:
The code is mostly correct.
### Suggestions to Improve:
- **Error Handling**: Add a check for missing or NaN values in the dataset to prevent errors when using `pivot()`.
- **Validation**: Confirm the data types are accurate (integers for IDs and floats for ratings).
- **Dependencies**: Ensure the environment has the required dependencies (`tensorflow`, `surprise`).

### Updated Version (with error handling and validation):
```python
import pandas as pd
import numpy as np
import tensorflow as tf
from surprise import Dataset, Reader

# Example dataset
ratings_data = {
    'user_id': [1, 2, 3, 4, 5],
    'movie_id': [1, 2, 3, 4, 5],
    'rating': [5, 4, 3, 4, 5],
    'timestamp': [978300760, 978301968, 978302344, 978300275, 978824291]
}

# Create DataFrame
ratings = pd.DataFrame(ratings_data)

# Drop unnecessary columns
ratings.drop('timestamp', axis=1, inplace=True)

# Ensure no missing values
ratings.dropna(inplace=True)

# Ensure correct data types
ratings['user_id'] = ratings['user_id'].astype(int)
ratings['movie_id'] = ratings['movie_id'].astype(int)
ratings['rating'] = ratings['rating'].astype(float)

# Preprocess for TensorFlow
def preprocess_for_tensorflow():
    # Create user-item matrix
    user_movie_matrix = ratings.pivot(index='user_id', columns='movie_id', values='rating').fillna(0)

    # Convert to TensorFlow tensors
    user_ids = tf.convert_to_tensor(user_movie_matrix.index.values)
    movie_ids = tf.convert_to_tensor(user_movie_matrix.columns.values)
    rating_matrix = tf.convert_to_tensor(user_movie_matrix.values, dtype=tf.float32)

    return user_ids, movie_ids, rating_matrix

# Preprocess for Surprise
def preprocess_for_surprise():
    # Define rating scale for Surprise
    reader = Reader(rating_scale=(1, 5))

    # Load dataset for Surprise
    surprise_data = Dataset.load_from_df(ratings[['user_id', 'movie_id', 'rating']], reader)

    return surprise_data

# Example usage
user_ids, movie_ids, rating_matrix = preprocess_for_tensorflow()
surprise_data = preprocess_for_surprise()

# Display first 5 entries
print(ratings.head())
print(user_ids[:5].numpy(), movie_ids[:5].numpy(), rating_matrix[:5].numpy())
```

### Key Improvements:
- **Missing Values**: `ratings.dropna()` ensures any missing data is removed.
- **Validation**: Data types are rechecked for safety.
- **Printing**: Added example print statements to show processed entries.

# Question 2 - Data Indexing

## 1.What is the value of p for this B+-tree? (Note that p is the order of a B+-tree)

## 2.Can you re-build a taller B+-tree with the same value of p using the same set of search-key values in the leaf nodes of the given tree? If yes, show the steps by drawing a new diagram whenever the height of the tree increases

![重建树图片](2.2.png "2.2")

## 3.Insert the search-key values 84, 19 and 32 in sequence to the given B+-tree, and draw a new diagram for each insertion

![插入生成树图片](2.3.png "2.3")

## 4.Suggest a sequence of search-key values to be deleted from the resultant B+-tree in Q2.2 to shrink the tree to 2 levels with the least number of deletions. Show the steps by drawing a new diagram whenever a node is deleted

![2.4](2.4.png "2.4")

# Question 3 - Data Querying

## 1.Write a query to print the hacker_id, name, and the number of distinct tasks each hacker participated in. Sort the result by the number of tasks in descending order, and then by hacker_id in ascending order if there’s a tie

In [ ]:
SELECT
    H.hacker_id,
    H.name,
    COUNT(DISTINCT S.task_id) AS task_count
FROM
    Hackers H
JOIN
    Submissions S ON H.hacker_id = S.hacker_id
GROUP BY
    H.hacker_id, H.name
ORDER BY
    task_count DESC,
    H.hacker_id ASC;

## 2.Write a query to find the task_id, description, and the total bonus awarded for each task. Sort the result by task_id in ascending order

In [ ]:
SELECT
    T.task_id,
    T.description,
    SUM(T.bonus) AS total_bonus
FROM
    Tasks T
JOIN
    (
        SELECT
            task_id,
            ROW_NUMBER() OVER (
                PARTITION BY task_id
                ORDER BY score DESC, submission_date, submission_id
            ) AS ranking
        FROM
            Submissions
    ) AS RankedSubmissions ON T.task_id = RankedSubmissions.task_id
WHERE
    RankedSubmissions.ranking <= 3
GROUP BY
    T.task_id, T.description
ORDER BY
    T.task_id ASC;

## 3.Write a query to list the submission_id, hacker_id, name, and score of the highest-scoring submission for each task submitted on 2023-01-01. If multiple submissions have the highest score for the same task, return the submission with the smalles  submission_id. Sort the result by task_id in ascending order

In [ ]:
SELECT
    S.submission_id,
    S.hacker_id,
    H.name,
    S.score
FROM
    Submissions S
JOIN
    Hackers H ON S.hacker_id = H.hacker_id
JOIN
    (
        SELECT
            task_id,
            MAX(score) AS max_score
        FROM
            Submissions
        WHERE
            submission_date = '2023-01-01'
        GROUP BY
            task_id
    ) AS MaxScores ON S.task_id = MaxScores.task_id
                   AND S.score = MaxScores.max_score
WHERE
    S.submission_date = '2023-01-01'
ORDER BY
    S.task_id ASC,
    S.submission_id ASC;

## 4.Write a query to print the hacker_id, name, and the total score each hacker achieved across all tasks. For each task, only the hacker's best score counts. Sort the result by total_score in descending order and by hacker_id in ascending order if there's a tie

In [ ]:
SELECT
    H.hacker_id,
    H.name,
    SUM(B.best_score) AS total_score
FROM
    Hackers H
JOIN
    (
        SELECT
            hacker_id,
            task_id,
            MAX(score) AS best_score
        FROM
            Submissions
        GROUP BY
            hacker_id, task_id
    ) AS B ON H.hacker_id = B.hacker_id
GROUP BY
    H.hacker_id, H.name
ORDER BY
    total_score DESC,
    H.hacker_id ASC;

## 5.Write a query to find the hacker_id, name, and bank_account of hackers who did not participate in any tasks

In [ ]:
SELECT
    H.hacker_id,
    H.name,
    H.bank_account
FROM
    Hackers H
LEFT JOIN
    Submissions S ON H.hacker_id = S.hacker_id
WHERE
    S.hacker_id IS NULL;

# Question 4 - Recommender System

## 1.Please write two basic approaches for recommender system and briefly explain them

- #### Collaborative filtering algorithm：
- The algorithm discovers user preferences by mining historical user behavior data, divides users into groups based on different preferences, and recommends products with similar tastes. Collaborative filtering recommendation algorithms are divided into two categories: neighborhood-based model algorithms, latent semantic analysis, and graph-based algorithms.
- #### Recommendation based on association rules：
- Association rules are used to discover potential associations between data, the most typical of which is shopping cart analysis in e-commerce. There are three important tree terms in association rules, namely support, confidence, and lift. Support is the frequency with which an item appears in all shopping carts. If we want to associate two items, then support is the frequency with which the two items appear at the same time. Support is used to mine the universality of relationships.

## 2.  One common challenge in recommender systems is the filter bubble problem. Explain what the filter bubble problem is and how it affects user experience. Suggest at least two strategies to mitigate the filter bubble issue

- ### What?
- The filter bubble problem refers to the system recommending content that matches the user's interests too centrally based on the user's historical behavior and preferences, which leads to a gradual narrowing of the information and perspectives received by the user, forming a state of "information isolation". In the filter bubble, the user is limited to the existing scope of interest and rarely comes into contact with diverse information or different perspectives.
- ### Influence
- Users can only see content that is highly consistent with their own preferences, resulting in a single source of information, which may lose the opportunity to explore new interests or new things. Filter bubbles will exacerbate users' existing biases or cognitive misunderstandings, because the recommendation system only provides information that is consistent with users' existing ideas, further strengthening existing views, which may affect the quality of users' decisions. Being in a filter bubble for a long time may cause users to get tired of recommendations of similar content, lack of freshness, and reduce user activity and interest in the platform.
- ### Solution
- Serendipity:
- 
Introduce some "accidental" recommendations regularly to expose users to some unexpected but attractive content. For example, the recommendation system can periodically introduce some new content or hot spots to increase the chances of users discovering new interest 
This method prevents users from falling into a fixed content cycle by intentionally "perturbing" the rommend deconten
- Transparency and personalized control:
- 
Provide users with options to control recommended content, such as allowing users to adjust the type of recommended content, set preference ranges, or even block specific types of content. This allows users to more actively manage and adjust their information flo .
It can also help users understand the basis for recommendations by regularly displaying transparency reports on recommendation algorithms, reducing the lack of trust caused by filter bubbles.t.

## 3.Rating prediction is an important task for a recommender system. Try to implement a recommendation model using the Goodbooks-10k dataset to predict user ratings. You have two options based on your available compute power

### (a)Option 1 (Full Dataset): For students with sufficient compute power, use the entire dataset (6 million ratings for 10,000 books by 53,000 users)

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics.pairwise import cosine_similarity
from math import sqrt

In [5]:
ratings = pd.read_csv('ratings.csv') 
books = pd.read_csv('books.csv')

In [30]:
train_data, test_data = train_test_split(ratings, test_size=0.2, random_state=42)

train_data = train_data.groupby(['user_id', 'book_id']).agg({'rating': 'mean'}).reset_index()

train_user_item_matrix = train_data.pivot(index='user_id', columns='book_id', values='rating')

train_user_item_matrix = train_user_item_matrix.fillna(np.nan)

user_similarity = cosine_similarity(train_user_item_matrix.fillna(0))
user_similarity = pd.DataFrame(user_similarity, index=train_user_item_matrix.index, columns=train_user_item_matrix.index)

MemoryError: Unable to allocate 21.0 GiB for an array with shape (53057, 53057) and data type float64

In [32]:
def predict_rating(user_id, book_id, k=10):
    if user_id not in train_user_item_matrix.index or book_id not in train_user_item_matrix.columns:
        return np.nan
    
    similar_users = user_similarity[user_id].sort_values(ascending=False)[1:k+1]

    ratings_sum = 0
    similarity_sum = 0
    for sim_user, similarity in similar_users.items():
        rating = train_user_item_matrix.at[sim_user, book_id]
        if not np.isnan(rating):
            ratings_sum += similarity * rating
            similarity_sum += abs(similarity)
    
    if similarity_sum == 0:
        return np.nan
    
    return ratings_sum / similarity_sum


def evaluate_model(test_data):
    predictions = []
    actuals = []
    for _, row in test_data.iterrows():
        user_id, book_id, actual_rating = row['user_id'], row['book_id'], row['rating']
        predicted_rating = predict_rating(user_id, book_id)
        
        if not np.isnan(predicted_rating):
            predictions.append(predicted_rating)
            actuals.append(actual_rating)
    
    mae = mean_absolute_error(actuals, predictions)
    rmse = sqrt(mean_squared_error(actuals, predictions))
    return mae, rmse

In [34]:
mae, rmse = evaluate_model(test_data)
with open('/mnt/data/evaluation_results.txt', 'w') as f:
    f.write(f"MAE: {mae}\n")
    f.write(f"RMSE: {rmse}\n")

KeyboardInterrupt: 